# Project 1 : Loading data on Sparkify

<img src="https://wiki.postgresql.org/images/a/a4/PostgreSQL_logo.3colors.svg" width="250" height="250">

# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
from datetime import datetime

In [2]:
conn = psycopg2.connect("host=127.0.0.1 port=30432 dbname=sparkifydb user=sparkify password=sparkify")
#conn.set_session(autocommit=True)
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [4]:
song_files = [cf for cf in get_files('data/song_data/') if "ipynb" not in cf]

In [5]:
!cat "{song_files[0]}"|jq . 

{
  "num_songs": 1,
  "artist_id": "AR7G5I41187FB4CE6C",
  "artist_latitude": null,
  "artist_longitude": null,
  "artist_location": "London, England",
  "artist_name": "Adam Ant",
  "song_id": "SONHOTT12A8C13493C",
  "title": "Something Girls",
  "duration": 233.40363,
  "year": 1982
}


In [6]:
df_song_file = []

for cf in song_files:
    df_song_file += [pd.read_json(cf,lines=True)]
    
df_song_file = pd.concat(df_song_file).reset_index().drop('index',1)

In [7]:
df_song_data = df_song_file[['song_id','title','artist_id','year','duration']]
df_song_data

,song_id,title,artist_id,year,duration
0,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
1,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
2,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608
3,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
4,SOQHXMF12AB0182363,Young Boy Blues,ARGSJW91187B9B1D6B,0,218.77506
...,...,...,...,...,...
66,SOLEYHO12AB0188A85,Got My Mojo Workin,ARAGB2O1187FB3A161,0,338.23302
67,SOGVQGJ12AB017F169,Ten Tonne,AR62SOJ1187FB47BB5,2005,337.68444
68,SOFFKZS12AB017F194,A Higher Place (Album Version),ARBEBBY1187B9B43DB,1994,236.17261
69,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),ARNNKDK1187B98BBD5,0,407.37914


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
def song_table_insert(cur, data):
    # Insert row
    try: 
        cur.execute(
        """
        INSERT INTO songs (song_id, title, artist_id, year, duration) \
        VALUES (%s,%s,%s,%s,%s)
        """, list(data.values))
        cur.connection.commit()
        return True
    except psycopg2.Error as e:
        print("Error: Could not create a row")
        print(e)
        cur.connection.rollback()
        
        return False

In [9]:
#df_song_data.apply(lambda d: song_table_insert(cur,d))
df_song_data.apply(lambda d: song_table_insert(cur,d),1)

0     True
1     True
2     True
3     True
4     True
      ... 
66    True
67    True
68    True
69    True
70    True
Length: 71, dtype: bool

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
df_artist_data = df_song_file[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]

In [11]:
len(set(df_artist_data['artist_id'].tolist()))

69

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
def artist_table_insert(cur, data):
    # Insert row
    try: 
        cur.execute(
        """
        INSERT INTO artists (artist_id, name, location, latitude, longitude) \
        VALUES (%s,%s,%s,%s,%s)
        """, list(data.values))
        cur.connection.commit()
        return True
    except psycopg2.Error as e:
        print(f"Error: Could not create a row: {data.values}")
        print(e)
        cur.connection.rollback()
        
        return False

In [13]:
df_artist_data.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
1,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
2,ARXR32B1187FB57099,Gob,,NaN,NaN
3,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
4,ARGSJW91187B9B1D6B,JennyAnyKind,North Carolina,35.21962,-80.01955


In [14]:
df_artist_data.apply(lambda d: artist_table_insert(cur,d),1)
#artist_table_insert(cur,np.ndarray([None, '','','','','']))

Error: Could not create a row: ['ARNTLGG11E2835DDB9' 'Clp' '' nan nan]
duplicate key value violates unique constraint "artists_pkey"
DETAIL:  Key (artist_id)=(ARNTLGG11E2835DDB9) already exists.

Error: Could not create a row: ['ARD7TVE1187B99BFB1' 'Casual' 'California - LA' nan nan]
duplicate key value violates unique constraint "artists_pkey"
DETAIL:  Key (artist_id)=(ARD7TVE1187B99BFB1) already exists.



0     True
1     True
2     True
3     True
4     True
      ... 
66    True
67    True
68    True
69    True
70    True
Length: 71, dtype: bool

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = [cf for cf in get_files('data/log_data/') if "ipynb" not in cf]

In [16]:
!cat "{log_files[0]}"|head -n 1|jq .

{
  "artist": "Frumpies",
  "auth": "Logged In",
  "firstName": "Anabelle",
  "gender": "F",
  "itemInSession": 0,
  "lastName": "Simpson",
  "length": 134.47791,
  "level": "free",
  "location": "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",
  "method": "PUT",
  "page": "NextSong",
  "registration": 1541044398796,
  "sessionId": 455,
  "song": "Fuck Kitty",
  "status": 200,
  "ts": 1541903636796,
  "userAgent": "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36\"",
  "userId": "69"
}


In [17]:
df_log_file = []

for cf in log_files:
    df_log_file += [pd.read_json(cf,lines=True)]
    
df_log_file = pd.concat(df_log_file).reset_index().drop('index',1).query("page == 'NextSong'")
df_log_file.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
df_log_file['ts_date'] = df_log_file['ts'].apply(lambda x: datetime.fromtimestamp(x / 1000))

column_labels = list(('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'))

time_df = pd.DataFrame(
    df_log_file['ts_date'].apply(lambda x: (x, x.hour, x.day, x.week, x.month, x.year, x.weekday())).tolist(),
    columns = column_labels
)
# ts is reduced by 1000; it makes easier that duplicates appear
#time_df = time_df.drop_duplicates()

In [19]:
time_df.head(10)
#2018-11-11 05:34:01.796000

,start_time,hour,day,week,month,year,weekday
0,2018-11-11 03:33:56.796,3,11,45,11,2018,6
1,2018-11-11 03:36:10.796,3,11,45,11,2018,6
2,2018-11-11 03:40:34.796,3,11,45,11,2018,6
3,2018-11-11 05:36:13.796,5,11,45,11,2018,6
4,2018-11-11 05:36:46.796,5,11,45,11,2018,6
5,2018-11-11 10:56:28.796,10,11,45,11,2018,6
6,2018-11-11 11:00:47.796,11,11,45,11,2018,6
7,2018-11-11 11:04:07.796,11,11,45,11,2018,6
8,2018-11-11 11:08:28.796,11,11,45,11,2018,6
9,2018-11-11 11:12:00.796,11,11,45,11,2018,6


In [20]:
df_log_file.head(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,ts_date
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 03:33:56.796
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 03:36:10.796
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 03:40:34.796
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,2018-11-11 05:36:13.796
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75,2018-11-11 05:36:46.796
6,Gary Allan,Logged In,Ryann,F,0,Smith,259.83955,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,The One,200,1541930188796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 10:56:28.796
7,Miracle Fortress,Logged In,Ryann,F,1,Smith,200.90730,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Five Roses,200,1541930447796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 11:00:47.796
8,Don Omar,Logged In,Ryann,F,2,Smith,261.35465,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Cuentale,200,1541930647796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 11:04:07.796
9,Jay-Z,Logged In,Ryann,F,3,Smith,212.27057,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,D'Evils,200,1541930908796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 11:08:28.796
10,Red Hot Chili Peppers,Logged In,Ryann,F,4,Smith,231.33995,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Easily (Album Version),200,1541931120796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 11:12:00.796


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
def time_table_insert(cur, data):
    # Insert row
    try: 
        cur.execute(
        """
        INSERT INTO time (start_time, hour, day, week, month, year, weekday) \
        VALUES (%s,%s,%s,%s,%s,%s,%s)
        """, list(data.values))
        cur.connection.commit()
        return True
    except psycopg2.Error as e:
        print("Error: Could not create a row")
        print(e)
        cur.connection.rollback()
        
        return False

In [22]:
time_df.apply(lambda d: time_table_insert(cur,d),1)

Error: Could not create a row
duplicate key value violates unique constraint "time_pkey"
DETAIL:  Key (start_time)=(2018-11-23 15:41:51.796) already exists.

Error: Could not create a row
duplicate key value violates unique constraint "time_pkey"
DETAIL:  Key (start_time)=(2018-11-14 05:53:36.796) already exists.

Error: Could not create a row
duplicate key value violates unique constraint "time_pkey"
DETAIL:  Key (start_time)=(2018-11-27 18:28:50.796) already exists.

Error: Could not create a row
duplicate key value violates unique constraint "time_pkey"
DETAIL:  Key (start_time)=(2018-11-15 19:55:04.796) already exists.

Error: Could not create a row
duplicate key value violates unique constraint "time_pkey"
DETAIL:  Key (start_time)=(2018-11-28 17:36:15.796) already exists.

Error: Could not create a row
duplicate key value violates unique constraint "time_pkey"
DETAIL:  Key (start_time)=(2018-11-28 20:59:23.796) already exists.

Error: Could not create a row
duplicate key value vi

0       True
1       True
2       True
3       True
4       True
        ... 
6815    True
6816    True
6817    True
6818    True
6819    True
Length: 6820, dtype: bool

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [23]:
user_df = df_log_file[['userId','firstName','lastName','gender','level']]

In [24]:
user_df = user_df.drop(user_df.query('userId == ""').index)

In [25]:
user_df['userId'] = user_df['userId'].apply(lambda x: int(x))

In [26]:
user_df = user_df.sort_values(by=['userId']).drop_duplicates()

In [27]:
user_df.query('userId == 15')

,userId,firstName,lastName,gender,level
2598,15,Lily,Koch,F,paid
6341,15,Lily,Koch,F,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [28]:
def user_table_insert(cur, data):
    # Insert row
    try: 
        cur.execute(
        """
        INSERT INTO users (user_id, first_name, last_name, gender, level) \
        VALUES (%s,%s,%s,%s,%s)
        """, list(data.values))
        cur.connection.commit()
        return True
    except psycopg2.Error as e:
        print(f"Error: Could not create a row: {data.values}")
        print(e)
        cur.connection.rollback()
        
        return False

In [29]:
user_df.apply(lambda d: user_table_insert(cur,d),1)

Error: Could not create a row: [15 'Lily' 'Koch' 'F' 'free']
duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(15) already exists.

Error: Could not create a row: [16 'Rylan' 'George' 'M' 'free']
duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(16) already exists.

Error: Could not create a row: [29 'Jacqueline' 'Lynch' 'F' 'free']
duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(29) already exists.

Error: Could not create a row: [36 'Matthew' 'Jones' 'M' 'free']
duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(36) already exists.

Error: Could not create a row: [49 'Chloe' 'Cuevas' 'F' 'free']
duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(49) already exists.

Error: Could not create a row: [80 'Tegan' 'Levine' 'F' 'free']
duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (user_id)=(80) a

6651    True
2080    True
1069    True
3952    True
128     True
        ... 
5914    True
2719    True
3720    True
6888    True
1185    True
Length: 104, dtype: bool

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [30]:
def song_select(cur, title, artist_name, duration):
    # Search for a song id
    try: 
        cur.execute(
        """
        SELECT songs.song_id, songs.artist_id
        FROM songs
        JOIN artists on songs.artist_id = artists.artist_id
        WHERE songs.title = %s
        AND artists.name = %s
        AND songs.duration = %s
        """, (title, artist_name, duration))

        res = cur.fetchone()
        if res is None:
            return pd.Series([None,None])
        else:
            return pd.Series(res)
    except psycopg2.Error as e:
        print(f"Error: Could not read a row")
        print(e)
        cur.connection.rollback()
        
        return pd.Series([None,None])

#song_select('Something Girls', 'Adam Ant', 233.40363)

In [31]:
def songplay_table_insert(cur, data):
    # Insert row
    try: 
        cur.execute(
        """
        INSERT INTO songplays (start_time, user_id, level, song_id, artist_id,session_id,location,user_agent) \
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        """, list(data.values))
        cur.connection.commit()
        return True
    except psycopg2.Error as e:
        print(f"Error: Could not create a row: {data.values}")
        print(e)
        cur.connection.rollback()
        
        return False

In [34]:
%%time
df_song_info = pd.DataFrame(df_log_file[['song','artist','length']].apply(lambda x: song_select(cur,x['song'],x['artist'],x['length']),1))
df_song_info.columns = ['song_id','artist_id']

CPU times: user 3.49 s, sys: 198 ms, total: 3.69 s
Wall time: 25.6 s


In [35]:
songsplay_df = df_log_file[['ts_date','userId','level','sessionId','location','userAgent']].merge(df_song_info,left_index=True, right_index=True)
songsplay_df = songsplay_df[['ts_date','userId','level','song_id','artist_id','sessionId','location','userAgent']]

In [36]:
%%time
songsplay_df.apply(lambda d: songplay_table_insert(cur,d),1)

CPU times: user 799 ms, sys: 384 ms, total: 1.18 s
Wall time: 25.3 s


0       True
1       True
2       True
4       True
5       True
        ... 
8051    True
8052    True
8053    True
8054    True
8055    True
Length: 6820, dtype: bool

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [37]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.